# Importing Libraries

In [39]:
import os
import warnings
import sys

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging

ModuleNotFoundError: No module named 'dvc'

In [8]:
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [10]:
df = pd.read_csv('../data/AdSmartABdata.csv')

# Exploaring The Categorical Columns

In [11]:
relevant_rows = df.query('yes == 1 | no == 0')
# relevant_rows

In [12]:
top2_platform = relevant_rows['platform_os'].value_counts().index.to_list()[:2]
top2_platform

[6, 5]

In [13]:
platform_list = []
for platform in top2_platform:
    platforms = relevant_rows.loc[relevant_rows['platform_os'] == platform]
    platform_list.append(platforms)
platform_six = platform_list[0]
platform_five = platform_list[1]


In [14]:
platform_six.to_csv('../data/platform_six.csv')
platform_five.to_csv('../data/platform_five.csv')

In [15]:
top2_browser = relevant_rows['browser'].value_counts().index.to_list()[:2]
top2_browser

['Chrome Mobile', 'Chrome Mobile WebView']

In [16]:
browser_list = []
for browser in top2_browser:
    browsers = relevant_rows.loc[relevant_rows['browser'] == browser]
    browser_list.append(browsers)
chrome_mobile_data = browser_list[0]
chrome_mobile_webview_data = browser_list[1]

In [17]:
chrome_mobile_data.to_csv('../data/chrome_mobile_data.csv')
chrome_mobile_webview_data.to_csv('../data/chrome_mobile_webview_data.csv')


In [33]:
df2 = relevant_rows.drop('no', axis=1)

In [34]:
df2.rename(columns = {'yes': 'clicked_or_not'}, inplace=True)

# Splitting Numerical and categorical variables

In [21]:
numerical_column = df2.select_dtypes(exclude="object").columns.tolist()
categorical_column = df2.select_dtypes(include="object").columns.tolist()
print("Numerical Columns:", numerical_column)
print("****************")
print("Categorical Columns:", categorical_column)

Numerical Columns: ['hour', 'platform_os', 'clicked_or_not']
****************
Categorical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']


# Splitting the columns for one hot encoding and label encoding

In [22]:
to_one_hot_encoding = [col for col in categorical_column if df2[col].nunique() <= 10 and df2[col].nunique() > 2]

# Get Categorical Column names thoose are not in "to_one_hot_encoding"
to_label_encoding = [col for col in categorical_column if not col in to_one_hot_encoding]

print("To One Hot Encoding:", to_one_hot_encoding)
print("To Label Encoding:", to_label_encoding)


To One Hot Encoding: ['date']
To Label Encoding: ['auction_id', 'experiment', 'device_make', 'browser']


In [23]:
# We will use built in pandas function "get_dummies()" to simply to encode "to_one_hot_encoding" columns
one_hot_encoded_columns = pd.get_dummies(df2[to_one_hot_encoding])
# one_hot_encoded_columns

In [24]:
le = LabelEncoder()
df2[to_label_encoding] = df2[to_label_encoding].apply(le.fit_transform)
# df2

In [25]:
# Label Encoding

label_encoded_columns = []
# For loop for each columns
for col in to_label_encoding:
    # We define new label encoder to each new column
    le = LabelEncoder()
    # Encode our data and create new Dataframe of it, 
    # notice that we gave column name in "columns" arguments
    column_dataframe = pd.DataFrame(le.fit_transform(df2[col]), columns=[col] )
    # and add new DataFrame to "label_encoded_columns" list
    label_encoded_columns.append(column_dataframe)

# Merge all data frames
label_encoded_columns = pd.concat(label_encoded_columns, axis=1)
# label_encoded_columns

In [26]:
# Copy our DataFrame to X variable
X = df2.copy()
X.drop(['date', 'browser'], axis=1, inplace=True)
# Merge DataFrames
X = pd.concat([X, one_hot_encoded_columns], axis=1)
print("All columns:", X.columns.tolist())
# X

All columns: ['auction_id', 'experiment', 'hour', 'device_make', 'platform_os', 'clicked_or_not', 'date_2020-07-03', 'date_2020-07-04', 'date_2020-07-05', 'date_2020-07-06', 'date_2020-07-07', 'date_2020-07-08', 'date_2020-07-09', 'date_2020-07-10']


In [35]:
# Define Y (This is the value we will predict)
y = X["yes"]

# Droping "class" from X
X.drop(["yes"], axis=1, inplace=True)
# X

KeyError: 'yes'

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.23)

NameError: name 'y' is not defined

# Logistic Regression

In [29]:
# Define Logistic Regression Model
log = LogisticRegression()
# We fit our model with our train data
log.fit(X_train, y_train)
# Then predict results from X_test data
pred_log = log.predict(X_test)

# See First 10 Predictions and They Actual Values
print("Predicted:", pred_log[0:10])
print("Actual:", y_test[0:10])

NameError: name 'X_train' is not defined

In [30]:
from matplotlib import pyplot
# get importance
importance = log.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

# Confusion Matrix

In [31]:
cm_log = confusion_matrix(y_test, pred_log)
print("Logistic Regression:\n", cm_log)

NameError: name 'confusion_matrix' is not defined

# Accuracey Score

In [32]:
# Logistic Regression
acc_log = accuracy_score(y_test, pred_log)
print("Logistic Regression accuracy:", acc_log)

NameError: name 'accuracy_score' is not defined